In [218]:
import numpy
import scipy.special
import matplotlib.pyplot

In [219]:
class neuralNetwork:
    def __init__(self, inputnode, hiddennode1, hiddennode2, outputnode, learningrate):
        self.inodes = inputnode
        self.hnodes1 = hiddennode1
        self.hnodes2 = hiddennode2
        self.onodes = outputnode
        self.lr = learningrate
        
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes1, self.inodes))
        self.wh1h2 = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes2, self.hnodes1))
        self.who = numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.onodes, self.hnodes2))
        
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    
    def train(self, input_list, target_list):
        inputs = numpy.array([input_list]).T
        targets = numpy.array([target_list]).T
        
        # Forward pass
        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        # Error Calculation
        output_error = targets - final_outputs
        hidden2_error = numpy.dot(self.who.T, output_error)
        hidden1_error = numpy.dot(self.wh1h2.T, hidden2_error)
        
        # Adjusting Weights
        self.who += self.lr * numpy.dot((output_error * final_outputs * (1 - final_outputs)), numpy.transpose(hidden2_outputs))
        self.wh1h2 += self.lr * numpy.dot((hidden2_error * hidden2_outputs * (1 - hidden2_outputs)), numpy.transpose(hidden1_outputs))
        self.wih += self.lr * numpy.dot((hidden1_error * hidden1_outputs * (1- hidden1_outputs)), numpy.transpose(inputs))
        
    
        pass
    
    
    def query(self, inputs_list):
        inputs = numpy.array([inputs_list]).T
        
        # Forward pass
        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        hidden2_outputs = self.activation_function(hidden2_inputs)
        
        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

## Network initialisieren

In [220]:
inputnode = 29
hiddennode1 = 500
hiddennode2 = 100
outputnode = 10
learningrate = 0.1

n = neuralNetwork(inputnode, hiddennode1, hiddennode2, outputnode, learningrate)

## Trainingsset einladen

In [221]:
training_data_file = open("all_data_oura_training.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

## Trainings Network

In [226]:
epoch = 10

for i in range(epoch):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 100 * 0.99) + 0.01

        # target matrix
        targets = numpy.zeros(outputnode) + 0.01
        x = (int(all_values[0]) - 50) // 5
        targets[x] = 0.99

        n.train(inputs, targets)
        
        pass
    pass

## Test Network

In [223]:
test_data_file = open("all_data_oura_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [224]:
scorecard = []

for record in test_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 100 * 0.99) + 0.01

        outputs = n.query(inputs)
        prediction = numpy.argmax(outputs)
        
        x = (int(all_values[0]) - 50) // 5
        if x == prediction:
            scorecard.append(1)
        else:
            scorecard.append(0)
        
        pass

In [225]:
scorecard_average = numpy.sum(scorecard) / len(scorecard)
print(scorecard_average)
print(scorecard)

0.5454545454545454
[1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
